In [1]:
install.packages("vetiver")
install.packages("plumber")

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependencies ‘cereal’, ‘pins’, ‘rapidoc’


Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependencies ‘webutils’, ‘sodium’, ‘swagger’




In [2]:
# This R environment comes with many helpful analytics packages installed
# It is defined by the kaggle/rstats Docker image: https://github.com/kaggle/docker-rstats
# For example, here's a helpful package to load

library(tidyverse) # metapackage of all tidyverse packages
library(caret) # modeling and machine learning
library(vetiver) # versionning, deployment of model
library(pins) # ressource sharing
library(plumber) # API creation
library(conflicted) # handling conflicts from imports
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

list.files(path = "../input")

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.2     ✔ readr     2.1.4
✔ forcats   1.0.0     ✔ stringr   1.5.0
✔ ggplot2   3.4.2     ✔ tibble    3.2.1
✔ lubridate 1.9.2     ✔ tidyr     1.3.0
✔ purrr     1.0.1     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors
Loading required package: lattice


Attaching package: ‘caret’


The following object is masked from ‘package:purrr’:

    lift


The following object is masked from ‘package:httr’:

    progress



Attaching package: ‘pins’


The following object is masked from ‘package:httr’:

    cache_info




[1] "custom-plumber"    "nasscds-accidents"

# Entraînements de modèle implémenté avec caret

## accident-glm.R de vignettes/experiments-deep-dive/accident-glm.R à partir de https://github.com/Azure/azureml-sdk-for-r

In [3]:
# Initialiser la graine pour la reproductibilité
set.seed(1234)
# Data from https://github.com/Azure/azureml-examples/tree/main
accidents <- readRDS(file.path("/kaggle/input/nasscds-accidents", "accidents.Rd"))
summary(accidents)

## Create data partition for use with caret
train.pct <- as.numeric("0.8")
if(length(train.pct)==0 || (train.pct<0) || (train.pct>1)) train.pct <- 0.75
accident_idx <- createDataPartition(accidents$dead, p = train.pct, list = FALSE)
accident_trn <- accidents[accident_idx, ]
accident_tst <- accidents[-accident_idx, ]
## utility function to calculate accuracy in test set
calc_acc = function(actual, predicted) {
  mean(actual == predicted)
}

## Caret GLM model on training set with 5-fold cross validation
accident_glm_mod <- train(
  form = dead ~ .,
  data = accident_trn,
  trControl = trainControl(method = "cv", number = 5),
  method = "glm",
  family = "binomial"
)
summary(accident_glm_mod)

output_dir = "outputs"
if (!dir.exists(output_dir)){
  dir.create(output_dir)
}
saveRDS(accident_glm_mod, file = "./outputs/model.rds")

    dead           dvcat         seatbelt           frontal      sex      
 alive:25036   1-9km/h:  686   belted:18572   notfrontal: 9351   f:12248  
 dead : 1180   10-24  :12847   none  : 7644   frontal   :16865   m:13968  
               25-39  : 8214                                              
               40-54  : 2977                                              
               55+    : 1492                                              
                                                                          
    ageOFocc        yearVeh        airbag        occRole     
 Min.   :16.00   Min.   :1953   airbag:14418   driver:20600  
 1st Qu.:22.00   1st Qu.:1989   none  :11798   pass  : 5616  
 Median :33.00   Median :1994                                
 Mean   :37.21   Mean   :1993                                
 3rd Qu.:48.00   3rd Qu.:1997                                
 Max.   :97.00   Max.   :2003                                


Call:
NULL

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-1.8986  -0.2559  -0.1449  -0.0904   3.4783  

Coefficients:
                 Estimate Std. Error z value Pr(>|z|)    
(Intercept)    -26.096791  18.539933  -1.408  0.15925    
dvcat.L          3.997358   0.454416   8.797  < 2e-16 ***
dvcat.Q          0.093744   0.384625   0.244  0.80744    
dvcat.C         -0.180786   0.239351  -0.755  0.45006    
`dvcat^4`        0.135593   0.114504   1.184  0.23634    
seatbeltnone     1.027883   0.076819  13.381  < 2e-16 ***
frontalfrontal  -1.112649   0.076316 -14.579  < 2e-16 ***
sexm             0.107204   0.076907   1.394  0.16334    
ageOFocc         0.034785   0.001908  18.230  < 2e-16 ***
yearVeh          0.010705   0.009281   1.153  0.24873    
airbagnone       0.203891   0.109969   1.854  0.06373 .  
occRolepass      0.229533   0.087558   2.621  0.00875 ** 
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

(Dispersion parameter for binomial fa

## Création de modèle vetiver déployable

In [4]:
v_mod <- vetiver_model(accident_glm_mod, model_name = "accident_model")

model_board <- board_folder(path = "/kaggle/working/pins-r", versioned = TRUE)
model_board %>% vetiver_pin_write(v_mod)

model_board %>% vetiver::vetiver_write_plumber("accident_model")
# pin_loc <- pins:::github_raw("DimbyTa/r-mlops-vetiver-huggingFace/main/pins-r/_pins.yaml")
# model_board <- board_url(pin_loc)
# model_board %>% vetiver::vetiver_write_plumber("accident_model")
write_board_manifest(model_board)

Creating new version '20231021T055644Z-ca45f'
Writing to pin 'accident_model'

Create a Model Card for your published model
• Model Cards provide a framework for transparent, responsible reporting
• Use the vetiver `.Rmd` template as a place to start
This message is displayed once per session.
Manifest file written to root folder of board, as `_pins.yaml`


## Création du fichier Docker pour le déploiement sur HuggingFace

In [5]:
vetiver::vetiver_write_docker(v_mod, "/kaggle/input/custom-plumber/plumber.R")

The following package(s) will be updated in the lockfile:

# CRAN -----------------------------------------------------------------------
- bit            [* -> 4.0.5]
- bit64          [* -> 4.0.5]
- bundle         [* -> 0.1.0]
- butcher        [* -> 0.3.2]
- caret          [* -> 6.0-94]
- cereal         [* -> 0.1.0]
- class          [* -> 7.3-22]
- cli            [* -> 3.6.1]
- clipr          [* -> 0.8.0]
- clock          [* -> 0.7.0]
- codetools      [* -> 0.2-19]
- colorspace     [* -> 2.1-0]
- cpp11          [* -> 0.4.4]
- crayon         [* -> 1.5.2]
- curl           [* -> 5.0.1]
- data.table     [* -> 1.14.8]
- diagram        [* -> 1.6.5]
- digest         [* -> 0.6.32]
- dplyr          [* -> 1.1.2]
- e1071          [* -> 1.7-13]
- ellipsis       [* -> 0.3.2]
- fansi          [* -> 1.0.4]
- farver         [* -> 2.1.1]
- foreach        [* -> 1.5.2]
- fs             [* -> 1.6.2]
- future         [* -> 1.33.0]
- future.apply   [* -> 1.11.0]
- generics       [* -> 0.1.3]
- ggplot2     